# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [14]:
# import libraries

import pandas as pd
import numpy as np
import sqlite3

from sqlalchemy import create_engine

import re
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
nltk.download('wordnet')
nltk.data.path.append('/Users/RobertHolzmann/nltk_data')


from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import regexp_tokenize




from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer


from xgboost import XGBClassifier
from gensim.models import Word2Vec



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/RobertHolzmann/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/RobertHolzmann/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/RobertHolzmann/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/RobertHolzmann/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
# load data from database

engine = create_engine('sqlite:////Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/data/DisasterResponse.db')


df = pd.read_sql('SELECT * FROM DisasterResponse', engine)



In [16]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Check the shape of the dataframe
df.shape

(26216, 40)

In [18]:
# Filter out any columns with only a single value 

valid_cols = [col for col in df.columns if df[col].nunique() > 1]

df = df[valid_cols]

In [19]:
df.shape

(26216, 39)

In [20]:
df['related'].value_counts()

related
1    19906
0     6122
2      188
Name: count, dtype: int64

In [21]:
# Some of the rows in the related column have values of '2'. 
# There weren't many, so I dropped them from the dataframe

df = df[df['related'] != 2]

In [22]:
df['related'].value_counts()

related
1    19906
0     6122
Name: count, dtype: int64

In [23]:
# Defining the feature variable X and target y

X = df['message']

y = df.iloc[:, 4:] 


In [24]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26211    The training demonstrated how to enhance micro...
26212    A suitable candidate has been selected and OCH...
26213    Proshika, operating in Cox's Bazar municipalit...
26214    Some 2,000 women protesting against the conduc...
26215    A radical shift in thinking came about as a re...
Name: message, Length: 26028, dtype: object

In [25]:
y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [26]:


def custom_tokenizer(text, url_place_holder_string="urlplaceholder"):

    """
    Tokenizes and preprocesses text by replacing URLs, tokenizing words, and lemmatizing tokens.

    Parameters
    ----------
    text : str
        The input text to be tokenized and preprocessed.
    url_place_holder_string : str, optional
        The string to replace URLs with (default is 'urlplaceholder').

    Returns
    -------
    list
        A list of clean tokens after URL replacement, tokenization, and lemmatization.
    """

    lemmatizer = WordNetLemmatizer()

    # Replace all URLs with a placeholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Tokenization using nltk's word_tokenize
    # tokens = word_tokenize(text)

    tokens = regexp_tokenize(text, pattern=r'\s|[\.,;!?()"]+', gaps=True)


    # Lemmatization and preprocessing
    clean_tokens = [lemmatizer.lemmatize(token.lower().strip()) for token in tokens]
    
    return clean_tokens




### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# Define the machine learning pipeline using FeatureUnion
pipeline = Pipeline([
    # 'features' step: combines multiple feature extraction processes
    ('features', FeatureUnion([
        # 'text_pipeline' step: processes text data
        ('text_pipeline', Pipeline([
            ('count_vectorizer', CountVectorizer(tokenizer=custom_tokenizer)),  # Custom tokenizer step that preprocesses text data
            ('tfidf_transformer', TfidfTransformer())  # Converts preprocessed text to TF-IDF features
        ]))
        # Note: You can add other feature processing steps in this FeatureUnion if needed
    ])),
    ('clf', MultiOutputClassifier(GradientBoostingClassifier()))  # The classifier predicts multiple target variables
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [28]:
# Split the data into training and testing sets
# X: Features (input data)
# y: Target variables (labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)




/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('count_vectorizer',
                                                                  CountVectorizer(tokenizer=<function custom_tokenizer at 0x1766c6700>)),
                                                                 ('tfidf_transformer',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=GradientBoostingClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [29]:
# Use the trained pipeline to predict the target variables for the test data 
y_pred = pipeline.predict(X_test)

# Evaluate the model by comparing predictions to the actual values
# Loop through each target variable (column) in y_test
for i, col in enumerate(y_test.columns):
    # Print the name of the current category being evaluated
    print(f"Category: {col}")
    
    # Generate and print the classification report for the current category
    # y_test[col]: Actual values for the current category
    # y_pred[:, i]: Predicted values for the current category
    # zero_division=1: Avoids division by zero errors by setting undefined metrics to 1
    print(classification_report(y_test[col], y_pred[:, i], zero_division=1))
    print("\n")  # Add a newline for better readability between categories

Category: related
              precision    recall  f1-score   support

           0       0.73      0.20      0.31      1247
           1       0.79      0.98      0.88      3959

    accuracy                           0.79      5206
   macro avg       0.76      0.59      0.60      5206
weighted avg       0.78      0.79      0.74      5206



Category: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      4304
           1       0.83      0.54      0.66       902

    accuracy                           0.90      5206
   macro avg       0.87      0.76      0.80      5206
weighted avg       0.90      0.90      0.89      5206



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           0.99      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg      

### 6. Improve your model
Use grid search to find better parameters. 

In [30]:

# This dictionary specifies the hyperparameters to tune and their respective values to try
params = {
    'clf__estimator__learning_rate': [0.01, 0.1, 0.2],  # Learning rate for the boosting process
    'clf__estimator__n_estimators': [50, 100, 200]      # Number of boosting stages (trees) in the model
}

# Initialize GridSearchCV with the pipeline and the defined parameter grid

grid_search = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='accuracy')

# Fit the GridSearchCV on the training data

grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV

print("Best Parameters:", grid_search.best_params_)


/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/

Best Parameters: {'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 200}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
# Retrieve the best model identified by GridSearchCV
best_model = grid_search.best_estimator_

# Use the best model to make predictions on the test set

y_pred = best_model.predict(X_test)

# Iterate over each column in the test set (y_test)

for i, col in enumerate(y_test.columns):
    # Print the category name
    print(f"Category: {col}")
    
    # Generate and print the classification report for each category
    print(classification_report(y_test[col], y_pred[:, i], zero_division=1))
    print("\n")


Category: related
              precision    recall  f1-score   support

           0       0.75      0.26      0.39      1247
           1       0.81      0.97      0.88      3959

    accuracy                           0.80      5206
   macro avg       0.78      0.62      0.64      5206
weighted avg       0.79      0.80      0.76      5206



Category: request
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      4304
           1       0.82      0.56      0.67       902

    accuracy                           0.90      5206
   macro avg       0.87      0.77      0.80      5206
weighted avg       0.90      0.90      0.90      5206



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       0.00      0.00      0.00        25

    accuracy                           0.99      5206
   macro avg       0.50      0.50      0.50      5206
weighted avg      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [32]:
# Trying out a Word2Vec model instead of TF-IDF

# Custom transformer class to convert text data into word embeddings using a pre-trained Word2Vec model
class Word2VecVectorizer(BaseEstimator, TransformerMixin):
    """
    A transformer class that converts tokenized text data into word embeddings using a pre-trained Word2Vec model.

    Attributes
    ----------
    model : gensim.models.Word2Vec
        The pre-trained Word2Vec model containing word embeddings.
    vector_size : int
        The dimensionality of the word vectors in the Word2Vec model.

    Methods
    -------
    fit(X, y=None):
        Returns the transformer itself, no fitting required.
    transform(X):
        Transforms the input data (tokenized text) into vector representations by averaging the word vectors for each document.
    """


    def __init__(self, model):
        """
        Initializes the Word2VecVectorizer with a pre-trained Word2Vec model.

        Parameters
        ----------
        model : gensim.models.Word2Vec
            The pre-trained Word2Vec model containing word embeddings.
        """
        
        self.model = model
        # Store the size of the word vectors from the Word2Vec model
        self.vector_size = model.wv.vector_size

    def fit(self, X, y=None):
        # No fitting required for this transformer, so we simply return self
        return self

    def transform(self, X):
        """
        Transforms each document in the input data into a vector representation.

        For each document (list of words), the method computes the average of the word vectors for the words found in the 
        Word2Vec model's vocabulary. If a word is not in the model's vocabulary, it is ignored. If none of the words in the 
        document are in the vocabulary, a zero vector of the appropriate size is returned.

        """
        return np.array([np.mean([self.model.wv[word] for word in words if word in self.model.wv]
                                 # Return a zero vector if the word is not found
                                 or [np.zeros(self.vector_size)], axis=0)
                         # Iterate over each document in X (assumed to be tokenized text)
                         for words in X])


In [33]:
from xgboost import XGBClassifier
from gensim.models import Word2Vec

# Trying out the Word2Vec in a pipeline along with XGBClassifier to improve model

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)


# Define the pipeline
pipeline_2 = Pipeline([
    ('features', FeatureUnion([
        ('count_vectorizer', CountVectorizer(tokenizer=custom_tokenizer)),  # Custom tokenizer step
        ('word2vec', Word2VecVectorizer(model=word2vec_model))  # Custom transformer for word embeddings
    ])),
    ('clf', MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')))
])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the pipeline
pipeline_2.fit(X_train, y_train)

# Make predictions
y_pred = pipeline_2.predict(X_test)

# Evaluate the model
for i, col in enumerate(y_test.columns):
    print(f"Category: {col}")
    print(classification_report(y_test[col], y_pred[:, i], zero_division=1))
    print("\n")


/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [20:24:12] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [20:24:15] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [20:24:18] WARNING: /Users/runner/work/xgboost/xgb

Category: related
              precision    recall  f1-score   support

           0       0.67      0.39      0.50      1247
           1       0.83      0.94      0.88      3959

    accuracy                           0.81      5206
   macro avg       0.75      0.67      0.69      5206
weighted avg       0.79      0.81      0.79      5206



Category: request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      4304
           1       0.80      0.58      0.67       902

    accuracy                           0.90      5206
   macro avg       0.86      0.77      0.81      5206
weighted avg       0.90      0.90      0.90      5206



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       1.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       1.00      0.50      0.50      5206
weighted avg      

In [34]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'clf__estimator__n_estimators': [50, 100, 200],  # Number of boosting rounds
    'clf__estimator__max_depth': [3, 5, 7],          # Maximum depth of a tree
    'clf__estimator__learning_rate': [0.01, 0.1, 0.2],  # Learning rate 
    'clf__estimator__subsample': [0.8, 1.0],         # Subsample ratio of the training instances
    'clf__estimator__colsample_bytree': [0.8, 1.0],  # Subsample ratio of columns when constructing each tree
}

grid_search = GridSearchCV(pipeline_2,param_grid, cv=3, n_jobs=-1, scoring='accuracy')

grid_search.fit(X_train, y_train)

# Best parameters found
print("Best parameters found:", grid_search.best_params_)

/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/Disaster/Disaster_Response/myenv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/RobertHolzmann/Downloads/

Best parameters found: {'clf__estimator__colsample_bytree': 0.8, 'clf__estimator__learning_rate': 0.2, 'clf__estimator__max_depth': 5, 'clf__estimator__n_estimators': 200, 'clf__estimator__subsample': 1.0}


In [35]:

# Retrieve the best model identified by GridSearchCV
best_model_xgb = grid_search.best_estimator_

# Use the best model to make predictions on the test set
y_pred = best_model_xgb.predict(X_test)

# Evaluate the model
for i, col in enumerate(y_test.columns):
    print(f"Category: {col}")
    print(classification_report(y_test[col], y_pred[:, i], zero_division=1))
    print("\n")

Category: related
              precision    recall  f1-score   support

           0       0.70      0.41      0.52      1247
           1       0.84      0.94      0.89      3959

    accuracy                           0.82      5206
   macro avg       0.77      0.68      0.70      5206
weighted avg       0.80      0.82      0.80      5206



Category: request
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      4304
           1       0.82      0.60      0.69       902

    accuracy                           0.91      5206
   macro avg       0.87      0.79      0.82      5206
weighted avg       0.90      0.91      0.90      5206



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5181
           1       1.00      0.00      0.00        25

    accuracy                           1.00      5206
   macro avg       1.00      0.50      0.50      5206
weighted avg      

### 9. Export your model as a pickle file

In [36]:
import os
from joblib import dump

# Define the file paths
joblib_file_path = 'classifier.joblib'

# Save the model using joblib if the file doesn't already exist
if not os.path.exists(joblib_file_path):
    dump(pipeline_2, joblib_file_path)
    print(f"Model saved as {joblib_file_path}")
else:
    print(f"File {joblib_file_path} already exists. Skipping save.")

Model saved as classifier.joblib


In [37]:
import pickle

pickle_file_path = 'classifier.pkl'

# Save the model using pickle if the file doesn't already exist
if not os.path.exists(pickle_file_path):
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(best_model, file)
    print(f"Model saved as {pickle_file_path}")
else:
    print(f"File {pickle_file_path} already exists. Skipping save.")

Model saved as classifier.pkl


### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.